In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
#import pandas as pd


In [4]:


target_X= pd.read_csv("../data/solor_train_input_7.csv").iloc[:,(1+24*0):].values
target_y =pd.read_csv("../data/solor_train_output_7.csv").iloc[:,1:].values
test_X= pd.read_csv("../data/solor_val_input_7.csv").iloc[:,(1+24*0):].values
test_y =pd.read_csv("../data/solor_val_output_7.csv").iloc[:,1:].values

X_train=target_X
y_train=target_y

backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

# 함수

In [6]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],y_train.shape[1],1,1,128

#################################################################################
# nbeats + I모델 생성 함수
def bulid_model(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK)
                   ,nb_blocks_per_stack=1, thetas_dim=(1,2,4,4),
                   share_weights_in_stack=True, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + G모델 생성 함수    
def bulid_model_G(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.GENERIC_BLOCK,NBeatsKeras.GENERIC_BLOCK)
                   ,nb_blocks_per_stack=5, thetas_dim=(4,4),
                   share_weights_in_stack=False, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models_G(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model_G(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################
# SMAPE 용
def train_bagging_models_smape(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

##########################################################################################
# 트랜스퍼 레이어
class PositionalEncoding(layers.Layer):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = layers.Dropout(rate=dropout)

        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = pe[np.newaxis, ...]

        self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, x):
        x = x + self.pe[:, :tf.shape(x)[1], :]
        return self.dropout(x)
##########################################################################################
# 트랜스퍼 레이어
def create_model(fn,d_model, nlayers, nhead, dropout, iw, ow,lr):
    
    
    x = layers.Dense(d_model // 2, activation='relu')(pretrained_output_reshaped)
    x = layers.Dense(d_model, activation='relu')(x)
    
    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)
    
    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)
    
    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)
    
    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)
    
    optimizer = Adam(learning_rate=lr)
    target_model = Model(inputs=inputs, outputs=outputs)
    target_model.compile(optimizer=optimizer, loss=fn)
    
    return target_model
#################################################################################
# 예측

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

# 모형적합

In [7]:
model_num = 10

mase_models = train_bagging_models(model_num, MASE(y_train,24),300,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',300,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),300,10,8,0.001)

2024-09-06 16:12:22.871278: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-06 16:12:22.871314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-09-06 16:12:22.872683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/300
72/72 [==============================] - 2s 10ms/step - loss: 1.2357 - val_loss: 0.7832
Epoch 2/300
72/72 [==============================] - 0s 4ms/step - loss: 0.8658 - val_loss: 0.7338
Epoch 3/300
72/72 [==============================] - 0s 3ms/step - loss: 0.8185 - val_loss: 0.7083
Epoch 4/300
72/72 [==============================] - 0s 3ms/step - loss: 0.7856 - val_loss: 0.6501
Epoch 5/300
72/72 [==============================] - 0s 3ms/step - loss: 0.7665 - val_loss: 0.6436
Epoch 6/300
72/72 [==============================] - 0s 4ms/step - loss: 0.7348 - val_loss: 0.6242
Epoch 7/300
72/72 [==============================] - 0s 3ms/step - loss: 0.7127 - val_loss: 0.6859
Epoch 8/300
72/72 [==============================] - 0s 3ms/step - loss: 0.7128 - val_loss: 0.6630
Epoch 9/300
72/72 [==============================] - 0s 3ms/step - loss: 0.6909 - val_loss: 0.6420
Epoch 10/300
72/72 [==============================] - 0s 3ms/step - loss: 0.6799 - val_loss: 0.6495
Epoch 11

In [8]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models

smape_predictions = bagging_predict2(pred1, test_X)
mase_predictions = bagging_predict2(pred2, test_X)
mape_predictions = bagging_predict2(pred3, test_X)
concat_I = np.concatenate([smape_predictions, mase_predictions,mape_predictions],axis=0)
fin_pred_I = np.median(concat_I,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
mean_squared_error(test_y.flatten(),fin_pred_I.flatten()),mean_absolute_error(test_y.flatten(),fin_pred_I.flatten())

12/12 [==============================] - 0s 975us/step


(0.22204233455907127, 0.24363793993498115)

In [12]:
pd.DataFrame(fin_pred_I.reshape(-1,24)).to_csv("../result7_new/NBEATs_B/pred_mid_I.csv")
for i in range(10):
    pd.DataFrame(concat_I[i].reshape(-1,24)).to_csv(f"../result7_new/NBEATs_B/pred_I{i}.csv")

# 일반블락

In [5]:
model_num = 10

mase_models_G = train_bagging_models_G(model_num, MASE(y_train,24),300,10,8,0.001)
mape_models_G = train_bagging_models_G(model_num,'mape',300,10,8,0.001)
smape_models_G = train_bagging_models_G(model_num, SMAPE(),300,10,8,0.001)

2024-08-30 13:34:23.283580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-30 13:34:23.283624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-08-30 13:34:23.284210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/300
72/72 [==============================] - 2s 8ms/step - loss: 1.0933 - val_loss: 0.7103
Epoch 2/300
72/72 [==============================] - 0s 5ms/step - loss: 0.8303 - val_loss: 0.6923
Epoch 3/300
72/72 [==============================] - 0s 5ms/step - loss: 0.8062 - val_loss: 0.6695
Epoch 4/300
72/72 [==============================] - 0s 5ms/step - loss: 0.7675 - val_loss: 0.6504
Epoch 5/300
72/72 [==============================] - 0s 5ms/step - loss: 0.7564 - val_loss: 0.6403
Epoch 6/300
72/72 [==============================] - 0s 5ms/step - loss: 0.7356 - val_loss: 0.6313
Epoch 7/300
72/72 [==============================] - 0s 4ms/step - loss: 0.7371 - val_loss: 0.7413
Epoch 8/300
72/72 [==============================] - 0s 4ms/step - loss: 0.7363 - val_loss: 0.6210
Epoch 9/300
72/72 [==============================] - 0s 4ms/step - loss: 0.7088 - val_loss: 0.6454
Epoch 10/300
72/72 [==============================] - 0s 4ms/step - loss: 0.6880 - val_loss: 0.6325
Epoch 11/

In [8]:
pred1,_=smape_models_G
pred2,_=mase_models_G
pred3,_=mape_models_G

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2, test_X)
mape_predictions_G = bagging_predict2(pred3, test_X)
concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
mean_squared_error(test_y.flatten(),fin_pred_G.flatten()),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())


12/12 [==============================] - 0s 1ms/step


(0.2244488404387937, 0.24109146095902362)

In [8]:

pd.DataFrame(fin_pred_G.reshape(-1,24)).to_csv("../result7_new/NBEATs_B/pred_mid_G.csv")
for i in range(10):
    pd.DataFrame(concat_G[i].reshape(-1,24)).to_csv(f"../result7_new/NBEATs_B/pred_G{i}.csv")